## Извлечение коллокаций и терминов
### Татьяна Шаврина
### Задание 1А

#### Импортируем необходимые библиотеки:

In [1]:
import pandas as pd
import os
import re
import nltk
from nltk.collocations import *
from nltk.metrics.distance import edit_distance 
from nltk.stem import WordNetLemmatizer
import wikiwords

#### Преобразуем данные в формат датафрейма для удобства

In [2]:
lst = []
keywords=[]
texts=[]
for file in os.listdir(r'/home/mi_air/Documents/HSE/corpus_linguistics/SemEval2010-Maui/maui-semeval2010-train'):
    lst.append(file)
for file in sorted(lst):
    if "key" in file:
        f = open(r'/home/mi_air/Documents/HSE/corpus_linguistics/SemEval2010-Maui/maui-semeval2010-train/'+file, 'r', encoding='utf8').read()
        keywords.append(re.split( '\n', f))
    else:
        f = open(r'/home/mi_air/Documents/HSE/corpus_linguistics/SemEval2010-Maui/maui-semeval2010-train/'+file, 'r', encoding='utf8').read()
        texts.append(f)

In [3]:
lst2 = []
keywords2=[]
texts2=[]
for file in os.listdir(r'/home/mi_air/Documents/HSE/corpus_linguistics/SemEval2010-Maui/maui-semeval2010-test'):
    lst2.append(file)
for file in sorted(lst2):
    if "key" in file:
        f = open(r'/home/mi_air/Documents/HSE/corpus_linguistics/SemEval2010-Maui/maui-semeval2010-test/'+file, 'r', encoding='utf8').read()
        keywords2.append(re.split( '\n', f))
    else:
        f = open(r'/home/mi_air/Documents/HSE/corpus_linguistics/SemEval2010-Maui/maui-semeval2010-test/'+file, 'r', encoding='utf8').read()
        texts2.append(f)

In [4]:
data = pd.DataFrame(
    {'text': texts,
     'keywords': keywords
    })
data.head()

,keywords,text
0,"[adaptive resource management, distributed rea...",Evaluating Adaptive Resource Management for\nD...
1,"[ensemble kalman filter, datum assimilation me...",Demonstration of Grid-Enabled Ensemble Kalman ...
2,"[multi-sequence positioning, wireless sensor n...",MSP: Multi-Sequence Positioning of Wireless Se...
3,"[wireless sensor network, localization, corner...",StarDust: A Flexible Architecture for Passive ...
4,"[sensor datum, separation of datum, datum sepa...",TSAR: A Two Tier Sensor Storage Architecture U...


In [5]:
testdata = pd.DataFrame(
    {'text': texts2,
     'keywords': keywords2
    })
testdata.head()

,keywords,text
0,"[grid service discovery, uddi, distributed web...",Scalable Grid Service Discovery Based on UDDI*...
1,"[target detection, sensor network, path exposu...",Sensor Deployment Strategy for Target Detectio...
2,"[voip conferencing system, packet-switched net...",Deployment Issues of a VoIP Conferencing Syste...
3,"[malware, swarm worm, emergent intelligence, s...",An Initial Analysis and Presentation of Malwar...
4,"[protocol framework, distributed algorithm, di...",Service Interface: A New Abstraction for Imple...


In [6]:
y_train = data["keywords"]
X_train= data["text"]
X_test = testdata["text"]
y_test = testdata["keywords"]

#### Введем список символов и стоп-слов, наличие которых будем считать критетрием для обраковки коллокаций

In [7]:
stopsymbols = list('\[\]9012345678(|){}+=_∗*−-&^%$#@!"/?.><,;:')
stopwords = ['the', 'a', 'of', 'in']
lemmatizer = WordNetLemmatizer()

#### Метрика оценки будет вот такая: берем золотой список терминов для данного текста, наш собственный список терминов, и поэлементно сравниваем, с учетом того, что совпадение строк может быть не полное, а на расстоянии левенштейна 3: (большие буквы, опечатки, сокращения - все то, что может случайно попасть). При таком совпадении засчитывается правильный ответ, и по их итоговому количеству выводится аккуратность. 

In [8]:
def accuracy(keywords, testkeywords):
    acc = 0
    for i in keywords:
        for y in testkeywords:
            try:
                if edit_distance(i, y, substitution_cost=1, transpositions=False)<=3:
                    acc+=1/len(testkeywords)
            except:
                pass
    return acc

#### Собственно извлечение коллокаций: для каждого текста проводится токенизация и лемматизация, стоп-слова и символы удаляются, затем ищутся частотные биграммы и триграммы, из них оставляются топ-20 (10+10) по likelihood_ratio. (Я пробовала также pmi, mi_like, raw_freq - но все mutual information поднимают наверх сочетания с редкими словами, а raw_freq слишком неустойчивая метрика)
#### Затем этот список ранжируется единым образом - создается единый список всех потенциальных терминов, они сортируются по частоте, и остается самый топ-10. Естественным образом, на данном этапе отсекается большая часть триграмм, так как частота триграмм обычно бывает ниже частоты биграмм, если только это не очень устойчивые сочетания, что нам и нужно!

#### Применяется также частеречный фильтр: если сочетание состоит из существительных или существительных и прилагательных, мы искусственно  поднимаем его частоту в 10 раз, что поднимает вероятность попадания сочетания в топ терминов

#### Также дополнительно ищется частота термина по википедии с помощью библиотеки wikiwords, и является множителем к частоте. Дополнительно проверяется, состоит ли строка из букв - если нет, то частота термина сильно понижается.

In [13]:
def get_wikifrequencies(candidate_keywords):
    max_frequency = float(wikiwords.freq('the'))
    return max([wikiwords.freq(w)/max_frequency for w in candidate_keywords])

In [23]:
bigram_measures = nltk.collocations.BigramAssocMeasures()
trigram_measures = nltk.collocations.TrigramAssocMeasures()
allacc=[]
for elem in range(len(X_test)):
    test = y_test[elem]
    l = 10
    X_test[elem] = X_test[elem].lower()
    elem = nltk.tokenize.casual.casual_tokenize(X_test[elem], preserve_case=True, reduce_len=False, strip_handles=False)
    elem = [lemmatizer.lemmatize(el) for el in elem if el not in stopsymbols]
    elem = [el for el in elem if el not in stopwords]
    finder2 = BigramCollocationFinder.from_words(elem)
    finder3 = TrigramCollocationFinder.from_words(elem)
    keystotest = []
    sp = [el for el in list(finder2.score_ngrams(bigram_measures.likelihood_ratio)[:10])]
    for el in list(finder3.score_ngrams(trigram_measures.likelihood_ratio)[:10]):
        sp.append(el)
    
    allkeys=dict(sp)
    newkeys = {}
    for i in sorted(allkeys):
            
        words = " ".join(i)
        if words.isalpha()==False:
            newkeys[i] = allkeys[i]/10
        pos = [n[1] for n in nltk.pos_tag(list(i))]
        nn = True
        for elem in pos:
            if 'NN' not in elem:
                if 'JJ' not in elem:
                    nn=False
        if nn:
            newkeys[i] = allkeys[i]*10
        
        newkeys[i] = allkeys[i]*get_wikifrequencies(words)
    
    
    for i in sorted(newkeys.items(), reverse=True)[:l]:

        words = " ".join(i[0])
        keystotest.append(words)
    a = accuracy(keystotest, test )
    allacc.append(a)
    print(a)
    

0.1
0.06666666666666667
0
0.2
0.4285714285714285
0.4999999999999999
0.06666666666666667
0.11764705882352941
0.13333333333333333
0.125
0.06666666666666667
0.05263157894736842
0
0
0.0625
0
0.3076923076923077
0
0
0.043478260869565216
0.2
0.08333333333333333
0.043478260869565216
0
0.26666666666666666
0.07692307692307693
0.21428571428571427
0.06666666666666667
0.07142857142857142
0
0.15000000000000002
0.05555555555555555
0.27272727272727276
0.3333333333333333
0
0.045454545454545456
0.13333333333333333
0.2
0.3076923076923077
0.125
0.14285714285714285
0.043478260869565216
0.16666666666666666
0.2631578947368421
0.05
0
0.05555555555555555
0.125
0.15384615384615385
0.08
0.07142857142857142
0.09090909090909091
0.15000000000000002
0
0.07692307692307693
0
0
0.13333333333333333
0.07692307692307693
0.13333333333333333
0.29411764705882354
0.1
0.047619047619047616
0
0.13333333333333333
0.23529411764705882
0.05555555555555555
0.1875
0.2727272727272727
0.15789473684210525
0
0.07142857142857142
0.41666666

#### Для каждого текста мы считали итоговую долю правильно выделенных терминов, и затем выводим средний результат:

In [24]:
import numpy
numpy.mean(allacc)

0.12589586319925866